In [1]:
import pymc3 as pm
import numpy as np
import theano.tensor as tt

In [2]:
# Initialize random number generator
np.random.seed(123)

# True parameter values
alpha, sigma = 1, 1
beta = [1, 2.5]

# Size of dataset
size = 100

# Predictor variable
X1 = np.random.randn(size)
X2 = np.random.randn(size) * 0.2

# Simulate outcome variable
data = alpha + beta[0]*X1 + beta[1]*X2 + np.random.randn(size)*sigma

In [11]:
from pymc3.distributions.dist_math import incomplete_beta, incomplete_beta_cfe, incomplete_beta_ps
def beta_logcdf(a, b, value):
    value = pm.floatX(tt.as_tensor(value))
    return tt.switch(
        tt.le(value, 0),
        -np.inf,
        tt.switch(
            tt.ge(value, 1),
            0,
            tt.log(incomplete_beta(a, b, value))
        )
    )

In [18]:
with pm.Model() as basic_model:
    # Priors for unknown model parameters
    a = pm.HalfNormal('alpha', sd=2)
    b = pm.HalfNormal('beta', sd=2)
    sigma = pm.HalfNormal('sigma', sd=1)

    m1 = beta_logcdf(a, b, 0.1)
    
    # Likelihood (sampling distribution) of observations
    Y_obs = pm.Normal('Y_obs', mu=m1, sd=sigma, observed=data)

In [19]:
basic_model.test_point

{'alpha_log__': array(0.46735583),
 'beta_log__': array(0.46735583),
 'sigma_log__': array(-0.22579135)}

In [20]:
basic_model.logp(basic_model.test_point)

array(-1509.42694554)

In [21]:
with basic_model:
    trace = pm.sample()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


AssertionError: 